In [1]:
import pandas as pd

### Load all video files for google 2022

In [2]:
vid = pd.read_csv("../01_prepare_ad_media/google2022_video_info.csv")

In [3]:
vid.head()

,filename,filepath,checksum_sha256
0,nIAa2wx0y2Q.mp4,/data/1/wesmediafowler/projects/AdMedia/Google...,3eb256475d44d124d8d671b15460066e767a87efaea137...
1,ZL_4kcX7kIM.mp4,/data/1/wesmediafowler/projects/AdMedia/Google...,16270b71732f94fd4a9bc2169f76cf94010d4e0bacbe1d...
2,QDt2eixQ-7E.mp4,/data/1/wesmediafowler/projects/AdMedia/Google...,2f5617ad11a6c8900a106261d9e56d5eb34c750c159312...
3,wAE3Adxg8z4.mp4,/data/1/wesmediafowler/projects/AdMedia/Google...,bc61eec68d979c23eacdee58c5df66a915c669d93283e8...
4,31E4r62s34c.mp4,/data/1/wesmediafowler/projects/AdMedia/Google...,d34b9f98fd5a1084181969b3ee61ebcd9f40bc79b2593f...


In [4]:
vid.shape

(25932, 3)

There is only one duplicate video. For convenience, we we copy all videos to the ./data/mp4

In [5]:
vid.loc[vid.duplicated(subset =['checksum_sha256'], keep=False)]

,filename,filepath,checksum_sha256
8838,c4BdVaD2IhM.mp4,/data/1/wesmediafowler/projects/AdMedia/Google...,1efed8286640916debd33876bbf358043e0ade4938d2c4...
16155,OLrW4v7tdYU.mp4,/data/1/wesmediafowler/projects/AdMedia/Google...,1efed8286640916debd33876bbf358043e0ade4938d2c4...


In [6]:
len(vid['checksum_sha256'].unique())

25931

In [7]:
len(vid['filename'].unique())

25932

### Load raw results

In [2]:
raw = pd.read_csv("result_asr_g2022_raw.csv")

In [3]:
raw.shape

(3, 3)

In [4]:
# Don't use .replace('_c', '') as it will mess up file names
raw['filename']=[str(i)[:-2]+'.mp4' for i in raw['filename']]

In [5]:
raw.head(2)

,filename,google_asr_text,stt_confidence
0,zFUv78n9yDA.mp4,Today we're making posole this crazy Max had n...,0.912839
1,YX0LgKik3kI.mp4,Vote for me Steve Fox. I did improve our child...,0.816703


In [6]:
vid = vid.merge(raw, on='filename', how='left')

NameError: name 'vid' is not defined

In [16]:
vid['google_asr_text'].isna().sum()

1026

In [17]:
vid.loc[vid['google_asr_text']==' '].shape

(377, 5)

In [18]:
vid['google_asr_status']='success'

In [19]:
vid.loc[vid['google_asr_text'].isna(), 'google_asr_status']='success_no_sound'
vid.loc[vid['google_asr_text']==' ', 'google_asr_status']='success_bgm'

In [20]:
vid.google_asr_status.value_counts(dropna=False)

success             24529
success_no_sound     1026
success_bgm           377
Name: google_asr_status, dtype: int64

In [22]:
vid['google_asr_model']='video_native'

In [25]:
vid=vid.rename(columns={'stt_confidence':'google_asr_confidence'})

In [27]:
del vid['filepath']

In [28]:
vid.head(2)

,filename,checksum_sha256,google_asr_text,google_asr_confidence,google_asr_status,google_asr_model
0,nIAa2wx0y2Q.mp4,3eb256475d44d124d8d671b15460066e767a87efaea137...,There's a difference between playing politics ...,0.804526,success,video_native
1,ZL_4kcX7kIM.mp4,16270b71732f94fd4a9bc2169f76cf94010d4e0bacbe1d...,,0.000000,success_bgm,video_native


In [29]:
vid.to_csv("result_asr_g2022.csv", index=False)